In [193]:
import finnhub as fh
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader as pdr
import quandl
import time
import datetime
from datetime import timezone
from math import nan
import scipy.stats
import statistics

## Data Gathering
Below are the functions used to gather price and financial statement data from YahooFinance, FinnHub and Robur Global using their respective APIs. 

In [2]:
# Pandas DF manipulation helper functions
def DateToUnix(date):
    return int(date.replace(tzinfo=timezone.utc).timestamp())

def UnixToDate(unix):
    return datetime.datetime.utcfromtimestamp(unix).strftime('%Y-%m-%d %H:%M:%S')

def FlattenStatementDF(statement_df):
    list_of_dict = list(statement_df.financials)
    return pd.DataFrame(list_of_dict)

def MergeStatementDF(statement_df_list):
    output = statement_df_list[0].merge(statement_df_list[1],on="period")
    output = output.merge(statement_df_list[2],on="period")
    output = output.set_index('period')
    return output

def SaveOutput(filename_prefix,**kwargs):
    for df_name, df in kwargs.items():
        filename = filename_prefix + '_' + df_name + '.csv'
        df.to_csv(filename)
        
def GetFailedCompanies(failed_ticker_list,ticker_company_table,reason_failed):
    print("Couldn't get",reason_failed,"data for",len(failed_ticker_list),"companies.")
    output = pd.DataFrame(columns=['ticker','company','reason_failed'])
    for ticker in failed_ticker_list:
        company = ticker_company_table.company.loc[ticker_company_table.finnhub == ticker]
        row = {'ticker':ticker,'company':company,'reason_failed':reason_failed}
        output = output.append(row,ignore_index=True)
    return output

In [14]:
# FinnHub
finnhub_client = fh.Client(api_key="br4je57rh5r8ufeothr0")

def GetFinnHubPrice(client,ticker,start_unix,end_unix):
    candles = client.stock_candles(ticker, 'D', start_unix, end_unix)
    if candles['s'] == 'no_data':
        print("Could not find price data for",ticker)
        return None
    else:
        return pd.DataFrame(candles)

def GetFinnHubFinancials(client,ticker,frequency,backtest_start_date):
    def _GetRelevantStatementIndex_(statement_df,backtest_start_date):
        # Flag if date of statement is within backtesting period
        def CheckDate(date_to_check,date_window):
            date_to_check = datetime.strptime(date_to_check, '%Y-%m-%d')
            return date_to_check >= date_window
        
        index = 0
        while index < len(statement_df):
            statement_date = statement_df.iloc[index][0]['period']
            if CheckDate(statement_date,backtest_start_date):
                index += 1
            else:
                return index + 1 # Include first statement out of period
        return index
    
    statement_type_list = ['bs','ic','cf']
    statement_df_list = []
    for statement_type in statement_type_list:
        statement_data_full = client.financials(ticker,statement_type,frequency)
        if statement_data_full['financials'] is None:
            print("Could not find financial data for",ticker)
            return None
        else:
            statement_data_full = pd.DataFrame(statement_data_full)
            statement_relevant_index = _GetRelevantStatementIndex_(statement_data_full,backtest_start_date)
            statement_data_relevant = statement_data_full.iloc[:statement_relevant_index]
            statement_df_list.append(FlattenStatementDF(statement_data_relevant))
    return MergeStatementDF(statement_df_list)

def GetISIN(ticker):
    company_profile = finnhub_client.company_profile(symbol=ticker)
    if len(company_profile) > 0:
        return company_profile['isin']
    else:
        return None

In [21]:
# Robur
def GetRoburCompanyCode(isin):
    robur_search = robur_codes_df.company_code.loc[robur_codes_df.isin_code == isin]
    if len(robur_search) > 0:
        return robur_search.iloc[0]
    else:
        return None

def GetRoburCompanyCode(ticker,robur_mapped_code_df):
    code_search = robur_mapped_code_df.robur_code.loc[robur_mapped_code_df.ticker == ticker]
    if len(code_search) > 0:
        return code_search.iloc[0]
    else:
        return None
    
def GetQuandlCode(company_code,financial_statement):
    return "RB1/{company_code}_HY{financial_statement}".format(company_code=company_code,financial_statement=financial_statement)

def DownloadRoburFinancials(company_code):
    statement_df_list = []
    for statement in robur_statements:
        quandl_code = GetQuandlCode(company_code,statement)
        statement_data = quandl.get(quandl_code,
                                    start_date = backtest_start_date,
                                    end_date = backtest_end_date)
        statement_df_list.append(statement_data)
    return MergeStatementDF(statement_df_list)

def MapRoburCodes(ticker_df):
    robur_code_mapping = pd.DataFrame(columns=['ticker','robur_code'])
    ticker_count = 0
    for ticker in ticker_df.finnhub:
        row = {}
        print("Checking",ticker)
        row['ticker'] = ticker
        isin = GetISIN(ticker)
        ticker_count += 1
        if (ticker_count+1) % 30 == 0:
            time.sleep(60)
        if isin is not None:
            robur_company_code = GetRoburCompanyCode(isin)
            if robur_company_code is not None:
                row['robur_code'] = robur_company_code
            else:
                row['robur_code'] = 'N/A'
        else:
            row['robur_code'] = 'N/A'
        robur_code_mapping = robur_code_mapping.append(row,ignore_index=True)
    return robur_code_mapping

In [213]:
# Factors class
class Company():
    def __init__(self,ticker,financials_df,share_price,data_source):
        self.share_price = share_price # Need to get latest price for share price
        self.ticker = ticker
        
        if data_source == 'fh':
            self.revenue = self.ValidateInput('revenue',financials_df)
            self.net_income = self.ValidateInput('netIncome',financials_df)
            self.total_assets = self.ValidateInput('totalAssets',financials_df)
            self.total_debt = self.ValidateInput('totalDebt',financials_df)
            self.shareholder_equity = self.total_assets - self.total_debt
            self.operating_cash_flow = self.ValidateInput('cashfromOperatingActivities',financials_df)
            self.ebitda = self.ValidateInput('netIncomeBeforeTaxes',financials_df)
            self.shares_outstanding = self.ValidateInput('totalCommonSharesOutstanding',financials_df)
            self.dividend_yield = self.ValidateInput('totalCashDividendsPaid',financials_df) / self.shares_outstanding
            self.market_cap = self.shares_outstanding * self.share_price
            self.cash_and_equivalents = self.ValidateInput('cash',financials_df) + self.ValidateInput('cashEquivalents',financials_df)
            self.enterprise_value = self.market_cap + self.total_debt + self.cash_and_equivalents
            self.book_value = self.shares_outstanding * self.ValidateInput('tangibleBookValueperShare',financials_df)
        elif data_source == 'rb':
            self.net_income = financials_df['Net Income exc. extra'],
            self.total_assets = financials_df['Total Assets'],
            self.total_debt = financials_df['Total Liabilities'],
            self.shareholder_equity = self.total_assets - self.total_debt,
            self.operating_cash_flow = financials_df['Cash from Operations'],
            self.ebitda = financials_df['Operating Income']
            self.shares_outstanding = financials_df['Diluted Shares OS']
            self.market_cap = self.shares_outstanding * share_price
            self.cash_and_equivalents = financials_df['End Cash']
            self.enterprise_value = self.market_cap + self.total_debt + self.cash_and_equivalents
            self.book_value = financials_df['Shareholder Equity']
        else:
            print("Data source not recognised")
            
    def ValidateInput(self,variable_name,input_df):
        input_df = input_df.fillna(0)
        return input_df[variable_name]
    
    def Divide(self,arg_1,arg_2):
        try:
            output = arg_1 / arg_2
        except ZeroDivisionError:
            output = nan
        return output
    
    def EquityQuality(self):
        metrics = {'return_on_equity' : [self.Divide(self.net_income, self.shareholder_equity)],
               'cash_flow_to_assets' : [self.Divide(self.operating_cash_flow, self.total_assets)],
               'debt_to_earnings' : [self.Divide(self.total_debt, self.ebitda)],
               'asset_leverage' : [self.Divide(self.total_debt, self.total_assets)]
               }
        return pd.DataFrame.from_dict(metrics)
    
    def EquitySize(self):
        metrics = {'market_cap' : [self.market_cap],
               'enterprise_value' : [self.enterprise_value],
               'total_assets' : [self.total_assets]
               }
        return pd.DataFrame(metrics)
    
    def EquityValue(self):
        metrics = {'dividend_yield' : [self.dividend_yield],
                   'earnings_to_price' : [(self.net_income / self.shares_outstanding) / self.share_price],
                   'book_to_price' : [self.Divide(self.book_value, self.shares_outstanding) / self.share_price],
#                    'sales_to_price' : [(self.revenue / self.shares_outstanding) / self.share_price],
                   'enterprise_to_ebitda' : [self.Divide(self.enterprise_value, self.ebitda)]
            }
        return pd.DataFrame(metrics)

In [210]:
# Factor calculation functions
def CalculateDailyFactors(ticker_list,unix_date,financials_df,price_df,robur_codes_df):
    
    quality_df = pd.DataFrame()
    value_df = pd.DataFrame()
    size_df = pd.DataFrame()
    for ticker in ticker_list:
        company_price = GetCompanyPriceFromDF(ticker,unix_date,price_df)
        company_share_price = company_price.o
        date_time = UnixToDate(unix_date)
        company_financials = GetCompanyFinancialsFromDF(ticker,date_time,financials_df)
        if company_financials is not None:
            company = Company(ticker,company_financials,company_share_price,'fh')
            try:
                company_quality = company.EquityQuality()
                company_quality['ticker'] = ticker
                quality_df = quality_df.append(company_quality)
            except:
                continue
                
            try:
                company_value = company.EquityValue()
                company_value['ticker'] = ticker
                value_df = value_df.append(company_value)
            except:
                continue
            
            try:
                company_size = company.EquitySize()
                company_size['ticker'] = ticker
                size_df = size_df.append(company_size)
            except:
                continue
        else:
            print("No financial data for",ticker)
            robur_company_code = GetRoburCompanyCode(ticker,robur_codes_df)
            if pd.isnull(robur_company_code):
                print("Could not find robur code")
            else:
                print("Retrieving robur financial data")
    return quality_df, value_df, size_df

def CalculateZScore(column_df):
    return stats.zscore(column_df)

def WeightFactorConstituents(factor_df):
    # Calculate z-scores for each indicator
    z_score_df = pd.DataFrame()
    for indicator_column in factor_df:
        indicator_z_score = CalculateZScore(factor_df[indicator_column])
        z_score_df[indicator_column] = indicator_z_score
    z_score_df = z_score_df.set_index(factor_df.index)
    
    # Calculate weighting for company based on z-scores of indicators
    z_score_df['summed_z_score'] = z_score_df.sum(axis=1)
    summed_z_score_mean = z_score_df['summed_z_score'].mean()
    summed_z_score_stdev = statistics.stdev(z_score_df['summed_z_score'])
    z_score_df['cdf'] = scipy.stats.norm(summed_z_score_mean, summed_z_score_stdev).cdf(z_score_df['summed_z_score'])
    z_score_df['weight'] = z_score_df['cdf'] / sum(z_score_df['cdf'])
    return z_score_df['weight']

def RankComponents(ticker_df,*args):
    max_length = max(len(t) for t in args)
    final_rank = np.zeros(max_length)
    for column in args:
        column_ranked = column.rank(numeric_only=True,na_option='keep',ascending=True)
        final_rank += column_ranked
    final_rank = final_rank / (max_length  + 1)
    return pd.DataFrame({'ticker':ticker_df,'rank':final_rank})

def RankQuality(quality_factor_df):
    max_length = len(quality_factor_df)
    roe_ranked = quality_factor_df.return_on_equity.rank(numeric_only=True,na_option='keep',ascending=True)
    cash_flow_to_assets_ranked = quality_factor_df.cash_flow_to_assets.rank(numeric_only=True,na_option='keep',ascending=True)
    debt_to_earnings_ranked = quality_factor_df.debt_to_earnings.rank(numeric_only=True,na_option='keep',ascending=False)
    asset_leverage_ranked = quality_factor_df.asset_leverage.rank(numeric_only=True,na_option='keep',ascending=False)
    final_rank = (roe_ranked + cash_flow_to_assets_ranked + debt_to_earnings_ranked + asset_leverage_ranked) / max_length
    return pd.DataFrame({'ticker':quality_factor_df['ticker'],'rank':final_rank})
        
def UpperDecile(df):
    upper_decile_rank = df.quantile(.9)['rank']
    output = df.loc[df['rank'] >= upper_decile_rank]
    return list(output['ticker'])

def LowerDecile(df):
    lower_decile_rank = df.quantile(.1)['rank']
    output = df.loc[df['rank'] <= lower_decile_rank]
    return list(output['ticker'])
    
def EmptyTimeSeriesDF():
    return pd.DataFrame(data=empty_master_df,columns=ticker_list,index=market_date_time_series)

def AssignWeights(weight_df,ticker_df,master_df,date):
    for ticker_idx in range(len(ticker_df)):
        ticker = ticker_df.iloc[ticker_idx]
        master_df[ticker][date] = weight_df.iloc[ticker_idx]

# Data extraction helper functions
def GetCompanyFinancialsFromDF(ticker,date_time,financial_df):
    released_financials_df = financial_df.loc[(financial_df.ticker == ticker) & (financial_df.period < date_time)]
    if len(released_financials_df) > 0:
        return released_financials_df.iloc[0]
    else:
        return None
    
def GetCompanyPriceFromDF(ticker,unix_date,price_df):
    return price_df.loc[(price_df.ticker == ticker) & (price_df.t == unix_date)].iloc[0].loc[['c','h','l','o','t','v']]

def GetLongestTimeSeriesTicker(ticker_list,price_df):
    length_ticker = {}
    for ticker in ticker_list:
        length_ticker.__setitem__(ticker, len(price_data.t.loc[price_data.ticker == ticker]))
    return max(length_ticker,key = length_ticker.get)

def GetDailyTickers(price_df,unix_date):
    return list(set(price_data.ticker.loc[price_data.t == unix_date]))

## Calculating Factors

In [12]:
# Initialisation
markets = ['HK','L','CO','MI','US']
investment_universe = pd.concat(pd.read_excel('investment_universe.xlsx', sheet_name=None), ignore_index=True)
robur_codes_map = pd.read_csv('robur_codes_mapped.csv')

In [219]:
quality_weight_markets = {}
value_weight_markets = {}
size_weight_markets = {}

for market in markets: #markets
    # Load data
    price_filename = './with_removals/' + market + '_price_data.csv'
    price_data = pd.read_csv(price_filename)
    # price_data.t = pd.to_datetime(price_data.t,unit='s')
    financials_filename = './with_removals/' + market + '_financial_data.csv'
    financials_data = pd.read_csv(financials_filename)
    financials_data.period = pd.to_datetime(financials_data.period,format='%Y-%m-%d')
    
    # Drop na's
    indexes_to_drop = financials_data[financials_data['commonStockTotal'] == 0].index
    indexes_to_drop.append(financials_data[financials_data['commonStockTotal'] == nan].index)
    financials_data.drop(indexes_to_drop, inplace=True)
    
    # Create master dataframe for entire time series
    ticker_list = list(set(financials_data.ticker))
    market_date_time_series = price_data.t.loc[price_data.ticker == GetLongestTimeSeriesTicker(ticker_list,price_data)]
    empty_master_df = np.zeros((len(market_date_time_series),len(ticker_list)))
    formatted_time_series = [UnixToDate(d) for d in market_date_time_series]
    
    # long df
#     master_quality_factor_time_series_long = pd.DataFrame(data=empty_master_df,columns=ticker_list,index=formatted_time_series)
# #     master_quality_factor_time_series_long.index = pd.to_datetime(master_quality_factor_time_series_long.index)
#     master_size_factor_time_series_long = pd.DataFrame(data=empty_master_df,columns=ticker_list,index=formatted_time_series)
# #     master_size_factor_time_series_long.index = pd.to_datetime(master_size_factor_time_series_long.index)
#     master_value_factor_time_series_long = pd.DataFrame(data=empty_master_df,columns=ticker_list,index=formatted_time_series)
# #     master_value_factor_time_series_long.index = pd.to_datetime(master_value_factor_time_series_long.index)
    
#     # short df
#     master_quality_factor_time_series_short = pd.DataFrame(data=empty_master_df,columns=ticker_list,index=formatted_time_series)
# #     master_quality_factor_time_series_short.index = pd.to_datetime(master_quality_factor_time_series_short.index)
#     master_size_factor_time_series_short = pd.DataFrame(data=empty_master_df,columns=ticker_list,index=formatted_time_series)
# #     master_size_factor_time_series_short.index = pd.to_datetime(master_size_factor_time_series_short.index)
#     master_value_factor_time_series_short = pd.DataFrame(data=empty_master_df,columns=ticker_list,index=formatted_time_series)
# #     master_value_factor_time_series_short.index = pd.to_datetime(master_value_factor_time_series_short.index)
    
    master_quality_factor_time_series = pd.DataFrame(columns=['date','ticker','weight','position'])
    master_size_factor_time_series_short = pd.DataFrame(columns=['date','ticker','weight','position'])
    master_value_factor_time_series_short = pd.DataFrame(columns=['date','ticker','weight','position'])
    
    # Get DF of financial data for single day of each stock
#     test_date = 1596697200 # DEBUG
#     test_date_list = [test_date] # DEBUG
    for date in market_date_time_series: # market_date_time_series
        date_formatted = UnixToDate(date)
        print("Calculating",date_formatted)
        # Calculate factor components
        daily_ticker_list = GetDailyTickers(price_data,date)
        quality_factor_df, value_factor_df, size_factor_df = CalculateDailyFactors(daily_ticker_list,date,financials_data,price_data,robur_codes_map)
        
        # Rank companies in each factor
        quality_ranked_df = RankQuality(quality_factor_df)
        value_ranked_df = RankComponents(value_factor_df.ticker,
                                         value_factor_df.dividend_yield,
                                         value_factor_df.earnings_to_price,
                                         value_factor_df.book_to_price,
#                                          value_factor_df.sales_to_price,
                                         value_factor_df.enterprise_to_ebitda)
        size_ranked_df = RankComponents(size_factor_df.ticker, size_factor_df.market_cap,size_factor_df.enterprise_value,size_factor_df.total_assets)
        
        # Get top and bottom companies by factor
        quality_long = quality_factor_df[quality_factor_df['ticker'].isin(UpperDecile(quality_ranked_df))]
        quality_short = quality_factor_df[quality_factor_df['ticker'].isin(LowerDecile(quality_ranked_df))]
        value_long = value_factor_df[value_factor_df['ticker'].isin(UpperDecile(value_ranked_df))]
        value_short = value_factor_df[value_factor_df['ticker'].isin(LowerDecile(value_ranked_df))]
        size_long = size_factor_df[size_factor_df['ticker'].isin(UpperDecile(size_ranked_df))]
        size_short = size_factor_df[size_factor_df['ticker'].isin(LowerDecile(size_ranked_df))]
        
        # Calculate factor investment weights
        quality_long_weight = WeightFactorConstituents(quality_long.iloc[:,:4])
        quality_short_weight = WeightFactorConstituents(quality_short.iloc[:,:4])
        value_long_weight = WeightFactorConstituents(value_long.iloc[:,:4])
        value_short_weight = WeightFactorConstituents(value_short.iloc[:,:4])
        size_long_weight = WeightFactorConstituents(size_long.iloc[:,:3])
        size_short_weight = WeightFactorConstituents(size_short.iloc[:,:3])
        
        # Assign weight to ticker in master df
        daily_quality_factor_time_series = pd.DataFrame(columns=['date','ticker','weight','position'])
        daily_size_factor_time_series = pd.DataFrame(columns=['date','ticker','weight','position'])
        daily_value_factor_time_series = pd.DataFrame(columns=['date','ticker','weight','position'])
        
        daily_quality_position_long = pd.DataFrame({'date':date_formatted,
                                                    'ticker':quality_long['ticker'],
                                                    'weight':quality_long_weight,
                                                    'position':'long'})
        daily_quality_position_short = pd.DataFrame({'date':date_formatted,
                                                    'ticker':quality_short['ticker'],
                                                    'weight':quality_short_weight,
                                                    'position':'short'})
        daily_size_position_long = pd.DataFrame({'date':date_formatted,
                                                    'ticker':size_long['ticker'],
                                                    'weight':size_long_weight,
                                                    'position':'long'})
        daily_size_position_short = pd.DataFrame({'date':date_formatted,
                                                    'ticker':size_short['ticker'],
                                                    'weight':size_short_weight,
                                                    'position':'short'})
        daily_value_position_long = pd.DataFrame({'date':date_formatted,
                                                    'ticker':value_long['ticker'],
                                                    'weight':value_long_weight,
                                                    'position':'long'})
        daily_value_position_short = pd.DataFrame({'date':date_formatted,
                                                    'ticker':value_short['ticker'],
                                                    'weight':value_short_weight,
                                                    'position':'short'})
        
        master_quality_factor_time_series = master_quality_factor_time_series.append(daily_quality_position_long.append(daily_quality_position_short))
        master_size_factor_time_series_short = master_size_factor_time_series_short.append(daily_size_position_long.append(daily_size_position_short))
        master_value_factor_time_series_short = master_value_factor_time_series_short.append(daily_value_position_long.append(daily_value_position_short))
        
    quality_weight_markets[market] = master_quality_factor_time_series
    value_weight_markets[market] = master_value_factor_time_series_short
    size_weight_markets[market] = master_size_factor_time_series_short
#         AssignWeights(quality_long_weight,quality_long['ticker'],master_quality_factor_time_series_long,date_formatted)
#         AssignWeights(value_long_weight,value_long['ticker'],master_value_factor_time_series_long,date_formatted)
#         AssignWeights(size_long_weight,size_long['ticker'],master_size_factor_time_series_long,date_formatted)
        
#         AssignWeights(quality_short_weight,quality_short['ticker'],master_quality_factor_time_series_short,date_formatted)
#         AssignWeights(value_short_weight,value_short['ticker'],master_value_factor_time_series_short,date_formatted)
#         AssignWeights(size_short_weight,size_short['ticker'],master_size_factor_time_series_short,date_formatted)
        
#         quality_factor_df, value_factor_df, size_factor_df = quality_factor_df.set_index('ticker'), value_factor_df.set_index('ticker'), size_factor_df.set_index('ticker')
        
        

Calculating 2019-07-01 07:00:00
No financial data for PPH.L
Retrieving robur financial data
No financial data for WWH.L
Could not find robur code
No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find robur code
No financial data for FEV.L
Could not find robur code


/Users/simondunkelman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  app.launch_new_instance()
/Users/simondunkelman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in double_scalars
/Users/simondunkelman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in double_scalars


No financial data for ATT.L
Could not find robur code
No financial data for SDP.L
Could not find robur code
No financial data for WIZZ.L
Retrieving robur financial data
Calculating 2019-07-02 07:00:00
No financial data for PPH.L
Retrieving robur financial data
No financial data for WWH.L
Could not find robur code
No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find robur code
No financial data for FEV.L
Could not find robur code
No financial data for ATT.L
Could not find robur code
No financial data for SDP.L
Could not find robur code
No financial data for WIZZ.L
Retrieving robur financial data
Calculating 2019-07-03 07:00:00
No financial data for PPH.L
Retrieving robur financial data
No financial data for WWH.L
Could not find robur code
No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find robur code
No financial data for FEV.L
Could not find robur code
No financial data for ATT.L
Could not find rob

No financial data for PPH.L
Retrieving robur financial data
No financial data for WWH.L
Could not find robur code
No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find robur code
No financial data for FEV.L
Could not find robur code
No financial data for ATT.L
Could not find robur code
No financial data for SDP.L
Could not find robur code
No financial data for WIZZ.L
Retrieving robur financial data
Calculating 2019-07-26 07:00:00
No financial data for PPH.L
Retrieving robur financial data
No financial data for WWH.L
Could not find robur code
No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find robur code
No financial data for FEV.L
Could not find robur code
No financial data for ATT.L
Could not find robur code
No financial data for SDP.L
Could not find robur code
No financial data for WIZZ.L
Retrieving robur financial data
Calculating 2019-07-29 07:00:00
No financial data for PPH.L
Retrieving robur f

No financial data for CTY.L
Could not find robur code
No financial data for FEV.L
Could not find robur code
No financial data for ATT.L
Could not find robur code
No financial data for SDP.L
Could not find robur code
No financial data for WIZZ.L
Retrieving robur financial data
Calculating 2019-08-20 07:00:00
No financial data for PPH.L
Retrieving robur financial data
No financial data for WWH.L
Could not find robur code
No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find robur code
No financial data for FEV.L
Could not find robur code
No financial data for ATT.L
Could not find robur code
No financial data for SDP.L
Could not find robur code
No financial data for WIZZ.L
Retrieving robur financial data
Calculating 2019-08-21 07:00:00
No financial data for PPH.L
Retrieving robur financial data
No financial data for WWH.L
Could not find robur code
No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find rob

...

No financial data for WWH.L
Could not find robur code
No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find robur code
No financial data for FEV.L
Could not find robur code
No financial data for ATT.L
Could not find robur code
No financial data for SDP.L
Could not find robur code
No financial data for WIZZ.L
Retrieving robur financial data
Calculating 2020-04-20 07:00:00
No financial data for PPH.L
Retrieving robur financial data
No financial data for CLSN.L
Could not find robur code
No financial data for WWH.L
Could not find robur code
No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find robur code
No financial data for FEV.L
Could not find robur code
No financial data for ATT.L
Could not find robur code
No financial data for SDP.L
Could not find robur code
No financial data for WIZZ.L
Retrieving robur financial data
Calculating 2020-04-21 07:00:00
No financial data for PPH.L
Retrieving robur financ

No financial data for ATT.L
Could not find robur code
No financial data for SDP.L
Could not find robur code
No financial data for WIZZ.L
Retrieving robur financial data
Calculating 2020-05-12 07:00:00
No financial data for PPH.L
Retrieving robur financial data
No financial data for CLSN.L
Could not find robur code
No financial data for WWH.L
Could not find robur code
No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find robur code
No financial data for FEV.L
Could not find robur code
No financial data for ATT.L
Could not find robur code
No financial data for SDP.L
Could not find robur code
No financial data for WIZZ.L
Retrieving robur financial data
Calculating 2020-05-13 07:00:00
No financial data for PPH.L
Retrieving robur financial data
No financial data for CLSN.L
Could not find robur code
No financial data for WWH.L
Could not find robur code
No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find r

No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find robur code
No financial data for FEV.L
Could not find robur code
No financial data for ATT.L
Could not find robur code
No financial data for SDP.L
Could not find robur code
No financial data for WIZZ.L
Retrieving robur financial data
Calculating 2020-06-04 07:00:00
No financial data for PPH.L
Retrieving robur financial data
No financial data for CLSN.L
Could not find robur code
No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find robur code
No financial data for FEV.L
Could not find robur code
No financial data for ATT.L
Could not find robur code
No financial data for SDP.L
Could not find robur code
No financial data for WIZZ.L
Retrieving robur financial data
Calculating 2020-06-05 07:00:00
No financial data for PPH.L
Retrieving robur financial data
No financial data for CLSN.L
Could not find robur code
No financial data for SOI.L
Could not find r

No financial data for ATT.L
Could not find robur code
No financial data for SDP.L
Could not find robur code
No financial data for WIZZ.L
Retrieving robur financial data
Calculating 2020-06-29 07:00:00
No financial data for PPH.L
Retrieving robur financial data
No financial data for CLSN.L
Could not find robur code
No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find robur code
No financial data for FEV.L
Could not find robur code
No financial data for ATT.L
Could not find robur code
No financial data for SDP.L
Could not find robur code
No financial data for WIZZ.L
Retrieving robur financial data
Calculating 2020-06-30 07:00:00
No financial data for PPH.L
Retrieving robur financial data
No financial data for CLSN.L
Could not find robur code
No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find robur code
No financial data for FEV.L
Could not find robur code
No financial data for ATT.L
Could not find r

No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find robur code
No financial data for FEV.L
Could not find robur code
No financial data for ATT.L
Could not find robur code
No financial data for SDP.L
Could not find robur code
No financial data for WIZZ.L
Retrieving robur financial data
Calculating 2020-07-23 07:00:00
No financial data for PPH.L
Retrieving robur financial data
No financial data for CLSN.L
Could not find robur code
No financial data for SOI.L
Could not find robur code
No financial data for CTY.L
Could not find robur code
No financial data for FEV.L
Could not find robur code
No financial data for ATT.L
Could not find robur code
No financial data for SDP.L
Could not find robur code
No financial data for WIZZ.L
Retrieving robur financial data
Calculating 2020-07-24 07:00:00
No financial data for PPH.L
Retrieving robur financial data
No financial data for CLSN.L
Could not find robur code
No financial data for SOI.L
Could not find r

,date,ticker,weight,position


In [196]:
quality_long = quality_factor_df[quality_factor_df['ticker'].isin(UpperDecile(quality_ranked_df))]
t = quality_long['ticker'].iloc[0]
formatted_time_series = [UnixToDate(d) for d in market_date_time_series]
# print(master_quality_factor_time_series_long[t][date_formatted])

# master_quality_factor_time_series_long
# AssignWeights(quality_long_weight,quality_long['ticker'],master_quality_factor_time_series_long,date_formatted)
# master_quality_factor_time_series_long
# quality_long_weight.iloc[0]

In [83]:
# RankQuality(quality_factor_df)
RankComponents(size_factor_df.market_cap,
               size_factor_df.enterprise_value,
               size_factor_df.total_assets)

0    2.769608
0    0.995098
0    0.725490
0    2.848039
0    1.588235
       ...   
0    1.990196
0    1.774510
0    1.784314
0    1.647059
0    2.598039
Name: return_on_equity, Length: 203, dtype: float64

In [52]:
# daily_data
z_score_test = WeightFactorConstituents(daily_data)
z_score_test

ticker
HILS.L    0.006785
HSX.L     0.002335
ASL.L     0.000015
PHP.L     0.007254
MDC.L     0.004613
            ...   
IWG.L     0.008333
WIZZ.L    0.006275
N91.L     0.003741
CNA.L     0.005327
IMI.L     0.006223
Name: weight, Length: 203, dtype: float64